In [ ]:
import altair as alt
import pandas as pd

- https://ggrepel.slowkow.com/articles/examples.html

In [ ]:
alt.renderers.set_embed_options(renderer="svg")

In [ ]:
DATA: str = "https://raw.githubusercontent.com/dssgPT/Plotting-Good-DSSG/main/desafios/002_Qualidade_do_Ar_em_Portugal_EEA/qualidade_do_ar.csv"
STATIONS: str = "./stations.csv"

In [ ]:
df = pd.read_csv(DATA, index_col=0)
df["cidade"] = df["cidade"].str.strip()

In [ ]:
cols = [
    "cidade",
    "poluente",
    "unidade_de_nivel_de_poluicao",
    "metrica",
    "nome_estacao",
    "ano",
    "nível_de_poluicao",
]

poluentes = ["O3", "NO2"]
year = 2021

df_year = df.query(
    "poluente == @poluentes & metrica == 'Média anual' & ano == @year"
).sort_values(by=["nome_estacao", "poluente"])[cols]

In [ ]:
df_year.shape

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df_year)

In [ ]:
df_year["nome_estacao"].nunique()

In [ ]:
# df_year["nome_estacao"].value_counts(dropna=False)

In [ ]:
df_pivot = df_year.pivot(
    index=["nome_estacao", "cidade"], columns="poluente", values="nível_de_poluicao"
)
df_pivot = df_pivot.reset_index().rename_axis(None, axis=1)
df_pivot = df_pivot.dropna(subset=["NO2", "O3"])

df_pivot.head()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df_pivot)

In [ ]:
df_pivot["max_NO2"] = df_pivot["NO2"].max()
df_pivot["min_NO2"] = df_pivot["NO2"].min()

df_pivot["max_O3"] = df_pivot["O3"].max()
df_pivot["min_O3"] = df_pivot["O3"].min()

df_pivot.head()

In [ ]:
df_max_min = df_pivot[
    (df_pivot["NO2"] == df_pivot["max_NO2"])
    | (df_pivot["NO2"] == df_pivot["min_NO2"])
    | (df_pivot["O3"] == df_pivot["max_O3"])
    | (df_pivot["O3"] == df_pivot["min_O3"])
]

df_max_min

In [ ]:
df_stations = pd.read_csv(STATIONS)

df_stations.head()

In [ ]:
df_max_min = df_max_min.merge(
    df_stations[
        ["v_estacoes_metadados.estacao_nome", "v_estacoes_metadados.concelho_nome"]
    ],
    left_on="nome_estacao",
    right_on="v_estacoes_metadados.estacao_nome",
)

df_max_min

In [ ]:
alt.Chart(df_pivot).mark_circle().encode(
    x=alt.X("NO2:Q", scale=alt.Scale(domain=[0, 32], nice=False)),
    y=alt.Y("O3:Q", scale=alt.Scale(domain=[0, 100])),
) + alt.Chart(df_max_min).mark_circle(color="red").encode(
    x=alt.X("NO2:Q"), y=alt.Y("O3:Q"), tooltip=["nome_estacao", "NO2", "O3"]
)

In [ ]:
# df_pivot.to_csv("scatterplot.csv", index=False)

---